In [22]:
import os
from dotenv import load_dotenv
load_dotenv()

from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

api_key = os.getenv("OPEN_ROUTER_API_KEY")

model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
    model="openai/gpt-4o-mini",    
    model_info={
        "family": "openai",
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "structured_output": True
    },
)

In [23]:
dsa_solver =  AssistantAgent(
    name="Complex_DSA_Solver",
    model_client = model_client,
    system_message="You are a helpful assistant that can solve complex DSA problems. give it in summary.",
    description="A complex DSA solver that can solve complex DSA problems.",
)

code_reviewer =  AssistantAgent(
    name="Code_Reviewer",
    model_client = model_client,
    system_message="You are a helpful assistant that can review code and provide feedback. give it in summary.",
    description="A code reviewer that can review code and provide feedback.",
)

code_editor =  AssistantAgent(
    name="Code_Editor",
    model_client = model_client,
    system_message="You are a helpful assistant that can edit code and provide feedback. give it in summary.",
    description="A code editor that can edit code and provide feedback.",
)

In [24]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage

team = RoundRobinGroupChat(
    participants=[dsa_solver, code_reviewer, code_editor],
    max_turns=3
)

In [25]:
import asyncio
async def run_team():
    task = TextMessage(content="Write a simple python code for palindrom of words", source="user")
    result = await team.run(task=task)
    print(result.messages[-1].content)
await run_team()


The code you provided for checking palindromes is indeed well-structured and effective. Here are the key highlights:

- **Functionality**: The `is_palindrome` function correctly identifies palindromic words by comparing the normalized (lowercase) version of the word with its reversed counterpart.
- **User Interaction**: The implementation includes an example that allows users to input a word and receive immediate feedback on whether it is a palindrome.
- **Clarity**: The use of slicing (`word[::-1]`) for reversal is a Pythonic approach and contributes to the code's readability.

In summary, the code is simple, succinct, and handles the palindrome check efficiently. It demonstrates good practices such as case normalization and user interaction. Great job!


# Since the Max turn is -> 1 the team will run only once from the DSA Solver and stops it after that

In [26]:
team = RoundRobinGroupChat(
    participants=[dsa_solver, code_reviewer, code_editor],
    max_turns=1
)

In [27]:
import asyncio
async def run_team():
    task = TextMessage(content="Write a simple python code for palindrom of words", source="user")
    result = await team.run(task=task)
    print(result.messages[-1].content)
await run_team()


Certainly! Here’s a straightforward Python program that checks if a given word is a palindrome:

```python
def is_palindrome(word):
    # Normalize the word by converting it to lowercase
    normalized_word = word.lower()
    # Check if the word reads the same forwards and backwards
    return normalized_word == normalized_word[::-1]

# Example usage
input_word = input("Enter a word: ")
if is_palindrome(input_word):
    print(f"{input_word} is a palindrome.")
else:
    print(f"{input_word} is not a palindrome.")
```

### Explanation
1. **Function Definition**: The function `is_palindrome(word)` checks if `word` is a palindrome.
2. **Normalization**: The word is converted to lowercase to ensure the check is case insensitive.
3. **Comparison**: The function compares the original normalized word with its reverse (`normalized_word[::-1]`).
4. **User Input**: The program prompts the user to enter a word and checks if it is a palindrome, printing the result.

You can run this code to test di

In [28]:
team = RoundRobinGroupChat(
    participants=[dsa_solver, code_reviewer, code_editor],
    max_turns=6
)

import asyncio
async def run_team():
    task = TextMessage(content="Write a simple python code for palindrom of words", source="user")
    result = await team.run(task=task)
    print(result)
await run_team()

messages=[TextMessage(id='c8fd5ef5-055e-4894-812d-6de82f9e3d00', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 34, 38, 504971, tzinfo=datetime.timezone.utc), content='Write a simple python code for palindrom of words', type='TextMessage'), TextMessage(id='051fef61-2ec0-416e-9c5f-eca4764381ca', source='Complex_DSA_Solver', models_usage=RequestUsage(prompt_tokens=465, completion_tokens=229), metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 34, 42, 716308, tzinfo=datetime.timezone.utc), content='Sure! Below is a simple Python code that checks if a given word is a palindrome:\n\n```python\ndef is_palindrome(word):\n    # Normalize the word by removing spaces and converting to lowercase\n    normalized_word = \'\'.join(word.split()).lower()\n    # Check if the word reads the same forwards and backwards\n    return normalized_word == normalized_word[::-1]\n\n# Example usage\ninput_word = input("Enter a word or phrase: ")\nif is_palindr

# There are differnet types of Termination one among them is:
# TextMention Termination, MaxMessage Termination

In [29]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination

my_termination = TextMentionTermination(text='TERMINATE') | MaxMessageTermination(max_messages=2)

team = RoundRobinGroupChat(
    participants=[dsa_solver, code_reviewer, code_editor],
    termination_condition=my_termination,
    max_turns=14
)

dsa_solver =  AssistantAgent(
    name="Complex_DSA_Solver",
    model_client = model_client,
    system_message="Your jobs is to solve complex DSA problems.",
)

code_reviewer =  AssistantAgent(
    name="Code_Reviewer",
    model_client = model_client,
    system_message="Yout job is to review the code. If the code is fine, Please say 'TERMINATE'",
)

code_editor =  AssistantAgent(
    name="Code_Editor",
    model_client = model_client,
    system_message="Your job is to edit the code. If the code Time complexity is more make it into O(n) or O(1) in the worst case as well",
)

async def run_team():
    task = TextMessage(content="Write a simple python code for addition of two numbers", source="user")
    result = await team.run(task=task)

    for each_agent in result.messages:
        print(f"{each_agent.source}: {each_agent.content}")
await run_team()

user: Write a simple python code for addition of two numbers
Complex_DSA_Solver: Sure! Here's a simple Python code to add two numbers:

```python
# Function to add two numbers
def add_two_numbers(num1, num2):
    return num1 + num2

# Get input from the user
try:
    number1 = float(input("Enter the first number: "))
    number2 = float(input("Enter the second number: "))
    
    # Calculate the sum
    sum_result = add_two_numbers(number1, number2)

    # Display the result
    print(f"The sum of {number1} and {number2} is: {sum_result}")
except ValueError:
    print("Please enter valid numbers.")
```

### How It Works:
1. **Function Definition**: The `add_two_numbers` function takes two numbers as parameters and returns their sum.
2. **User Input**: The program prompts the user to enter two numbers. It uses `float` to allow for decimal numbers.
3. **Error Handling**: The code includes a `try-except` block to handle cases where the input is not a valid number.
4. **Output**: Finally,

# the below code run_stream throws the output one be one as it is completed 

In [30]:
from autogen_agentchat.base import TaskResult
team2 = RoundRobinGroupChat(
    participants=[dsa_solver, code_reviewer, code_editor],
    termination_condition=my_termination,
    max_turns=14
)
async for message in team2.run_stream(task="Write a simple hello world! code?"):
    print(type(message))
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message.source, message)

<class 'autogen_agentchat.messages.TextMessage'>
user id='0582e40a-b75c-47a4-901f-28970621b83f' source='user' models_usage=None metadata={} created_at=datetime.datetime(2026, 2, 15, 11, 35, 13, 57018, tzinfo=datetime.timezone.utc) content='Write a simple hello world! code?' type='TextMessage'
<class 'autogen_agentchat.messages.TextMessage'>
Complex_DSA_Solver id='11ac70d5-ac40-480b-a4c3-cdd11dac06a2' source='Complex_DSA_Solver' models_usage=RequestUsage(prompt_tokens=30, completion_tokens=179) metadata={} created_at=datetime.datetime(2026, 2, 15, 11, 35, 16, 948931, tzinfo=datetime.timezone.utc) content='Certainly! Here are simple "Hello, World!" examples in a few different programming languages:\n\n### Python\n```python\nprint("Hello, World!")\n```\n\n### Java\n```java\npublic class HelloWorld {\n    public static void main(String[] args) {\n        System.out.println("Hello, World!");\n    }\n}\n```\n\n### JavaScript\n```javascript\nconsole.log("Hello, World!");\n```\n\n### C\n```c\n

# if i want to run the roundrobin for 2 times that means the termination max_messages = 7 (6 {teams} + 1 {user})

In [31]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.conditions import TextMentionTermination, MaxMessageTermination

my_termination = TextMentionTermination(text='TERMINATE') | MaxMessageTermination(max_messages=7)

team = RoundRobinGroupChat(
    participants=[dsa_solver, code_reviewer, code_editor],
    termination_condition=my_termination,
)

dsa_solver =  AssistantAgent(
    name="Complex_DSA_Solver",
    model_client = model_client,
    system_message="Your jobs is to solve complex DSA problems.",
)

code_reviewer =  AssistantAgent(
    name="Code_Reviewer",
    model_client = model_client,
    system_message="Yout job is to review the code. If the code is fine, Please say 'TERMINATE'",
)

code_editor =  AssistantAgent(
    name="Code_Editor",
    model_client = model_client,
    system_message="Your job is to edit the code. If the code Time complexity is more make it into O(n) or O(1) in the worst case as well",
)

async def run_team():
    task = TextMessage(content="Write a simple python code for addition of two numbers", source="user")
    result = await team.run(task=task)

    for each_agent in result.messages:
        print(f"{each_agent.source}: {each_agent.content}")
await run_team()

user: Write a simple python code for addition of two numbers
Complex_DSA_Solver: Sure! Here's a simple Python code snippet that adds two numbers:

```python
# Function to add two numbers
def add_numbers(num1, num2):
    return num1 + num2

# Input from the user
number1 = float(input("Enter the first number: "))
number2 = float(input("Enter the second number: "))

# Call the function and display the result
result = add_numbers(number1, number2)
print("The sum of", number1, "and", number2, "is:", result)
```

You can run this code in a Python environment. It will prompt you to enter two numbers and then display their sum.
Code_Reviewer: TERMINATE


In [32]:
state =  await team.save_state()
state

{'type': 'TeamState',
 'version': '1.0.0',
 'agent_states': {'Complex_DSA_Solver': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{'content': 'Write a simple hello world! code?',
       'source': 'user',
       'type': 'UserMessage'},
      {'content': 'Certainly! Here are simple "Hello, World!" examples in a few different programming languages:\n\n### Python\n```python\nprint("Hello, World!")\n```\n\n### Java\n```java\npublic class HelloWorld {\n    public static void main(String[] args) {\n        System.out.println("Hello, World!");\n    }\n}\n```\n\n### JavaScript\n```javascript\nconsole.log("Hello, World!");\n```\n\n### C\n```c\n#include <stdio.h>\n\nint main() {\n    printf("Hello, World!\\n");\n    return 0;\n}\n```\n\n### C++\n```cpp\n#include <iostream>\n\nint main() {\n    std::cout << "Hello, World!" << std::endl;\n    return 0;\n}\n```\n\n### Ruby\n```ruby\

In [33]:
agent1_agent_first =  AssistantAgent(
    name="agent1",
    model_client = model_client,
    system_message="add 1 to the number, first number is 0 and give the reuslt as ouput",
)

agent2_agent_second =  AssistantAgent(
    name="agent2",
    model_client = model_client,
    system_message="add 1 to the number, you got from the previous agent1 and give the reuslt as ouput",
)

agent3_agent_third =  AssistantAgent(
    name="agent3",
    model_client = model_client,
    system_message="add 1 to the number, you got from the previous agent3 and give the reuslt as ouput",
)

team3 = RoundRobinGroupChat(
    participants=[agent1_agent_first, agent2_agent_second, agent3_agent_third], max_turns=2
)

# the session is resumed from where it stopped

In [34]:
from autogen_agentchat.ui import Console

await Console(team3.run_stream())

---------- TextMessage (agent1) ----------
The result of adding 1 to the number 0 is 1.
---------- TextMessage (agent2) ----------
The result of adding 1 to the previous result (1) is 2.


TaskResult(messages=[TextMessage(id='ede8e81a-8a99-4a7c-a268-b776eff42610', source='agent1', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=15), metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 35, 21, 852722, tzinfo=datetime.timezone.utc), content='The result of adding 1 to the number 0 is 1.', type='TextMessage'), TextMessage(id='858ea7db-c8fb-480a-b081-762a78079dfb', source='agent2', models_usage=RequestUsage(prompt_tokens=52, completion_tokens=17), metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 35, 22, 696815, tzinfo=datetime.timezone.utc), content='The result of adding 1 to the previous result (1) is 2.', type='TextMessage')], stop_reason='Maximum number of turns 2 reached.')

In [35]:
from autogen_agentchat.ui import Console

await Console(team3.run_stream())

---------- TextMessage (agent3) ----------
The result of adding 1 to the previous result (2) is 3.
---------- TextMessage (agent1) ----------
The result of adding 1 to the previous result (3) is 4.


TaskResult(messages=[TextMessage(id='3625ce8c-bb7b-4b13-9916-247c733b77a2', source='agent3', models_usage=RequestUsage(prompt_tokens=76, completion_tokens=17), metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 35, 23, 720552, tzinfo=datetime.timezone.utc), content='The result of adding 1 to the previous result (2) is 3.', type='TextMessage'), TextMessage(id='d74a6886-097d-4835-83b3-3610a924eefc', source='agent1', models_usage=RequestUsage(prompt_tokens=95, completion_tokens=17), metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 35, 24, 888974, tzinfo=datetime.timezone.utc), content='The result of adding 1 to the previous result (3) is 4.', type='TextMessage')], stop_reason='Maximum number of turns 2 reached.')

In [36]:
from autogen_agentchat.ui import Console

await Console(team3.run_stream())

---------- TextMessage (agent2) ----------
The result of adding 1 to the previous result (4) is 5.
---------- TextMessage (agent3) ----------
The result of adding 1 to the previous result (5) is 6.


TaskResult(messages=[TextMessage(id='90e0dc54-31f3-4d33-b47b-a3c17c2cb96f', source='agent2', models_usage=RequestUsage(prompt_tokens=121, completion_tokens=17), metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 35, 26, 4182, tzinfo=datetime.timezone.utc), content='The result of adding 1 to the previous result (4) is 5.', type='TextMessage'), TextMessage(id='b7217813-a6cc-401e-8f59-762eb8006797', source='agent3', models_usage=RequestUsage(prompt_tokens=145, completion_tokens=17), metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 35, 27, 184497, tzinfo=datetime.timezone.utc), content='The result of adding 1 to the previous result (5) is 6.', type='TextMessage')], stop_reason='Maximum number of turns 2 reached.')

# after resting it

In [37]:
await team3.reset()
from autogen_agentchat.ui import Console

await Console(team3.run_stream())

---------- TextMessage (agent1) ----------
1
---------- TextMessage (agent2) ----------
2


TaskResult(messages=[TextMessage(id='856c8cc1-8619-48bf-8ade-9b1713e75bd9', source='agent1', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=1), metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 35, 27, 742295, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='f0b47180-8750-4db3-9961-35a6686aa9c4', source='agent2', models_usage=RequestUsage(prompt_tokens=38, completion_tokens=1), metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 35, 28, 923140, tzinfo=datetime.timezone.utc), content='2', type='TextMessage')], stop_reason='Maximum number of turns 2 reached.')

# Single Agent Team

In [38]:
team4 = RoundRobinGroupChat(
    participants=[agent1_agent_first], max_turns=5
)
await team4.reset()
await Console(team4.run_stream())

---------- TextMessage (agent1) ----------
1
---------- TextMessage (agent1) ----------
1
---------- TextMessage (agent1) ----------
1
---------- TextMessage (agent1) ----------
1
---------- TextMessage (agent1) ----------
1


TaskResult(messages=[TextMessage(id='497b90ba-1fa3-428b-ac79-b503dd2a0cf0', source='agent1', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=1), metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 35, 29, 571516, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='2e41df15-f185-4049-bda8-3d44e2cee57c', source='agent1', models_usage=RequestUsage(prompt_tokens=33, completion_tokens=1), metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 35, 30, 167603, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='718fc83a-3f9f-4bf6-9be4-8e17b1d5bcdb', source='agent1', models_usage=RequestUsage(prompt_tokens=38, completion_tokens=1), metadata={}, created_at=datetime.datetime(2026, 2, 15, 11, 35, 30, 713624, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='58fed73e-395f-402a-9e56-11f95e6e656a', source='agent1', models_usage=RequestUsage(prompt_tokens=43, completion_tokens=1), metadat

# Aborting a team -> cancellation_token.cancel()

In [39]:
from autogen_core import CancellationToken
# Create a cancellation token.
cancellation_token = CancellationToken()

# Use another coroutine to run the team.
run = asyncio.create_task(
    team4.run(
        task="Translate the poem to Spanish.",
        cancellation_token=cancellation_token,
    )
)

# Cancel the run.
cancellation_token.cancel()

try:
    result = await run  # This will raise a CancelledError.
except asyncio.CancelledError as e:
    print(e)
    print("Task was cancelled.")


Task was cancelled.
